In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-08-14 14:28:13--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  44.7MB/s    in 0.6s    

2025-08-14 14:28:14 (44.7 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# print(f'{df_ratings.head()}\n{df_books.head()}')
print(f'{df_ratings.shape}\n{df_books.shape}')

# df_ratings.info()

(1149780, 3)
(271379, 3)


In [ ]:
# df_books_ratings = pd.merge(df_books, df_ratings, how='inner', on='isbn')
# df_books_ratings.head()

To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

In [ ]:
# firstly to get good books
df_gb = df_ratings.groupby(['isbn']).count().reset_index()
good_books = df_gb[df_gb['rating'] >= 100]['isbn']

# df_gb.head()
good_books = df_books[df_books['isbn'].isin(good_books)]
good_books.shape

(727, 3)

In [ ]:
# to get the user with the certified rating now
df_gu = df_ratings[['user', 'rating']].groupby(['user']).count().reset_index()
valid_user = df_gu[df_gu['rating'] >= 200]['user']
# valid_user.shape
df = df_ratings[df_ratings['user'].isin(valid_user)]
df = df.loc[df["isbn"].isin(good_books["isbn"])]

df = df.drop_duplicates(subset=['isbn', 'user'])

In [ ]:
df.shape

(49517, 3)

In [ ]:

# Create pivot table (books as rows, users as columns)
book_user_matrix = df.pivot(
    index='isbn',
    columns='user',
    values='rating'
).fillna(0)

In [ ]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler(with_mean=True, with_std=False)
# book_user_matrix_scaled = scaler.fit_transform(book_user_matrix)

In [ ]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(csr_matrix(book_user_matrix))

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(title = ""):
    # to cross check my work i'll add this to debug my dfs creates
    try:
        book = good_books[good_books["title"] == title]
    except KeyError as e:
        print('Book', e, 'does not exist')
        return

    # preparing to ping the k neigbours
    book_details = book_user_matrix[book_user_matrix.index.isin(book["isbn"])]
    distance, indices = model.kneighbors(book_details.values.reshape(1, -1), n_neighbors=6)

    distance =  distance[0][1:]
    idx= indices[0][1:]

    titles = [
        good_books[good_books['isbn'] == book_user_matrix.iloc[i].name]["title"].values\
        for i in idx
    ]

    recommended = [list(z) for z in zip(titles, distance)][::-1]
    return [title, recommended]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [[array(["I'll Be Seeing You"], dtype=object), np.float32(0.8016211)], [array(['The Weight of Water'], dtype=object), np.float32(0.77085835)], [array(['The Surgeon'], dtype=object), np.float32(0.7699411)], [array(['I Know This Much Is True'], dtype=object), np.float32(0.7677075)], [array(['The Lovely Bones: A Novel'], dtype=object), np.float32(0.7234864)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
